In [2]:
%use s2, algoquant, krangl
import java.time.format.DateTimeFormatter

// select data from cache for specific dates
fun getDataByDate(dateList: List<String>, data: RandomAccessCache<Double>): BufferCache<Double> {
    val dataSelected = BufferCache<Double>()
    dateList.forEach { date ->
        val dateTime = LocalDate.parse(date).atStartOfDay()
        dataSelected.add(dateTime, data.get(dateTime).data())
    }
    return dataSelected
}

// Parameters
val begDateS = "2020-01-01"
val endDateS = "2020-06-01"
val stockTic = "AAPL"
val factorList = mutableListOf("roa_ttm", "ig_yoy", "quick_ratio")

// Rebalancing Period
val period = Period.ofWeeks(2)

val endDate = LocalDate.parse(endDateS)
val selectColList = factorList.toMutableList()
selectColList.add(0, "trade_date")
val selectCol = selectColList.toTypedArray()
val df = s2.dataFrame("factor-premium", begDateS, endDateS)
val dfSelect = df.select(*selectCol).sortedBy("trade_date")

// get stock price data
val stkPrc = s2.table("stock", stockTic, begDateS, endDateS)
val stkCache = BufferCache<Double>()
stkPrc.forEach { row ->
    stkCache.add(row.getDate("date").atStartOfDay(), row.getDouble("close"))
}

// find trading dates base on period
val selectDateList = mutableListOf<String>()
var tempDate = stkCache.first().time().toLocalDate()
while (tempDate < endDate) {
    selectDateList.add(tempDate.toString())
    tempDate = tempDate.plus(period)
}

// calculate stock period return
val selectStkPrc = getDataByDate(selectDateList, stkCache)
val stkRet = Cache2ReturnsMatrix(selectStkPrc)

// transform factor premiums to price in order to calculate period return
val premiumDataset = mutableListOf<SequentialCache<Double>>()
for (index1 in 0 until factorList.size) {
    val tempPremiumPrc = BufferCache<Double>()
    tempPremiumPrc.add((dfSelect["trade_date"][0] as LocalDate).atStartOfDay(),1.0)
    for (index2 in 1 until dfSelect.nrow) {
        val thisDateTime = (dfSelect["trade_date"][index2] as LocalDate).atStartOfDay()
        val prevPrc = tempPremiumPrc.get(thisDateTime).data()
        tempPremiumPrc.add(
            thisDateTime,
            prevPrc * (1 + (dfSelect[factorList[index1]][index2] as Double))
        )
    }
    val tempSelectPremiumPrc = getDataByDate(selectDateList, tempPremiumPrc)
    premiumDataset.add(tempSelectPremiumPrc)
}

// calculate period premium
val premiumRetMatrix = Cache2ReturnsMatrix(premiumDataset)

// transform stock return to vector
val retVector = DenseVector(stkRet)

// linear regression
val lmProblem = LMProblem(retVector, premiumRetMatrix, false)
val ols = OLSRegression(lmProblem)

// calculate expected return
var expectRet = 0.0
for (index in factorList.indices) {
    val factorExp = ols.beta().betaHat().get(index + 1)
    println("Exposure for factor ${factorList[index]} is: ${factorExp.toString()}")
    expectRet += factorExp * premiumRetMatrix.getColumn(index + 1)[premiumRetMatrix.nRows()]
}

println("Expected return of stock '$stockTic' " +
        "for period ${endDate.plusDays(1)} ~ ${endDate.plus(period).toString()} is: " +
        "${expectRet.toString()}")


15:05:47,364 |-INFO in ch.qos.logback.classic.LoggerContext[default] - Could NOT find resource [logback-test.xml]
15:05:47,364 |-INFO in ch.qos.logback.classic.LoggerContext[default] - Could NOT find resource [logback.groovy]
15:05:47,364 |-INFO in ch.qos.logback.classic.LoggerContext[default] - Found resource [logback.xml] at [jar:file:/home/jovyan/.ivy2/cache/tech.nmfin/algoquant-core/jars/algoquant-core-3.2.0-SNAPSHOT.jar!/logback.xml]
15:05:47,373 |-INFO in ch.qos.logback.core.joran.spi.ConfigurationWatchList@3ede5f5f - URL [jar:file:/home/jovyan/.ivy2/cache/tech.nmfin/algoquant-core/jars/algoquant-core-3.2.0-SNAPSHOT.jar!/logback.xml] is not of type file
15:05:47,404 |-INFO in ch.qos.logback.classic.joran.action.ConfigurationAction - debug attribute not set
15:05:47,404 |-INFO in ch.qos.logback.core.joran.action.AppenderAction - About to instantiate appender of type [ch.qos.logback.core.ConsoleAppender]
15:05:47,408 |-INFO in ch.qos.logback.core.joran.action.AppenderAction - Namin

java.lang.RuntimeException: table 'factor-premium' not found
dev.nm.s2.data.DataCommand.void(fb:32)
dev.nm.s2.data.DataCommand.void(fb:168)
dev.nm.s2.data.DataCommand.dataFrame(fb:1)
Line_11_jupyter.<init>(Line_11.jupyter.kts:27)
java.base/jdk.internal.reflect.NativeConstructorAccessorImpl.newInstance0(Native Method)
java.base/jdk.internal.reflect.NativeConstructorAccessorImpl.newInstance(NativeConstructorAccessorImpl.java:62)
java.base/jdk.internal.reflect.DelegatingConstructorAccessorImpl.newInstance(DelegatingConstructorAccessorImpl.java:45)
java.base/java.lang.reflect.Constructor.newInstance(Constructor.java:490)
kotlin.script.experimental.jvm.BasicJvmScriptEvaluator.evalWithConfigAndOtherScriptsResults(BasicJvmScriptEvaluator.kt:96)
kotlin.script.experimental.jvm.BasicJvmScriptEvaluator.invoke$suspendImpl(BasicJvmScriptEvaluator.kt:41)
kotlin.script.experimental.jvm.BasicJvmScriptEvaluator.invoke(BasicJvmScriptEvaluator.kt)
kotlin.script.experimental.jvm.BasicJvmReplEvaluator.eval